## Workspace Management

The corresponding functionality is now demonstrated through notebooks in the **Getting Started** section of the Workspace repository:  
- https://github.com/EOEPCA/workspace/blob/main/docs/getting-started  

This workspace repository also contains a declarative description of the **eoepca-demo** setup, including:

- the Kubernetes and Keycloak configuration (example users, clients, and roles) for the eoepca-demo cluster as well as
- references to the overall authentication and authorization model

Further details on the UAM concept can be found here:  
- https://github.com/EOEPCA/workspace/#authentication-and-user-management  

Additional documentation and related references are linked throughout the repository for deeper guidance.